In [1]:
import pandas as pd
import numpy as np
import re
import csv
from string import punctuation
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import accuracy_score
import pickle

In [2]:
tsv_file_path = "train_preprocess.tsv.txt"

data_text = []
label = []

with open(tsv_file_path, encoding='utf-8') as tsvfile:
    tsv_reader = csv.reader(tsvfile, delimiter='\t')
    for row in tsv_reader:
        data_text.append(row[0])
        label.append(row[1])

df = pd.DataFrame({'data_text': data_text, 'label': label})

In [3]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=0)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=0)

In [4]:
print(train_data.shape)
print(test_data.shape)
print(val_data.shape)

(7040, 2)
(2200, 2)
(1760, 2)


In [5]:
def lowercasing(paragraph):
    return paragraph.lower()

In [6]:
def menghilangkan_tandabaca(paragraph):
    new_paragraph = re.sub(fr'[{punctuation}]', r'', paragraph)
    return new_paragraph

In [7]:
def text_normalization(paragraph):
    paragraph = lowercasing(paragraph)
    paragraph = menghilangkan_tandabaca(paragraph)
    paragraph = re.sub(r"[ ]+",r' ',paragraph)
    return paragraph

In [8]:
train_data['data_text'] = train_data['data_text'].apply(lambda x: text_normalization(x))
test_data['data_text'] = test_data['data_text'].apply(lambda x: text_normalization(x))
val_data['data_text'] = val_data['data_text'].apply(lambda x: text_normalization(x))

In [9]:
tokenizer = Tokenizer(oov_token='<UNK>')

In [10]:
tokenizer.fit_on_texts(train_data['data_text'])

In [11]:
train_data_tf = tokenizer.texts_to_sequences(train_data['data_text'])
test_data_tf = tokenizer.texts_to_sequences(test_data['data_text'])
val_data_tf = tokenizer.texts_to_sequences(val_data['data_text'])

In [12]:
max_len = int(np.quantile([len(x) for x in train_data_tf], 0.9))

In [13]:
train_padded = pad_sequences(sequences = train_data_tf, padding='post', maxlen=max_len)
test_padded = pad_sequences(sequences = test_data_tf, padding='post', maxlen=max_len)
val_padded = pad_sequences(sequences = val_data_tf, padding='post', maxlen=max_len)

In [14]:
print(train_padded.shape)
print(test_padded.shape)
print(val_padded.shape)

(7040, 59)
(2200, 59)
(1760, 59)


In [15]:
onehot = OneHotEncoder()

In [16]:
onehot.fit_transform(train_data[['label']])

<7040x3 sparse matrix of type '<class 'numpy.float64'>'
	with 7040 stored elements in Compressed Sparse Row format>

In [17]:
train_labels = onehot.fit_transform(train_data[['label']]).toarray()

In [18]:
model = Sequential()

In [19]:
max_features = len(tokenizer.index_word)
batch_size=16
output_dim = 64
input_len = max_len

In [20]:
model = keras.Sequential()
model.add(layers.Embedding(input_dim=len(tokenizer.index_word)+1,output_dim=output_dim, input_length=input_len))
model.add(layers.LSTM(128, dropout=0.2))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

In [21]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [22]:
maximum_index_training = int(np.floor(train_padded.shape[0]/batch_size)*batch_size)
model.fit(x=train_padded[:maximum_index_training], 
          y=train_labels[:maximum_index_training],
          batch_size=batch_size, 
          epochs=20, 
          shuffle=False)

Epoch 1/20
440/440 [==============================] - 19s 39ms/step - loss: 0.8283 - accuracy: 0.6381
Epoch 2/20
440/440 [==============================] - 17s 39ms/step - loss: 0.7438 - accuracy: 0.6855
Epoch 3/20
440/440 [==============================] - 17s 39ms/step - loss: 0.8902 - accuracy: 0.5881
Epoch 4/20
440/440 [==============================] - 17s 39ms/step - loss: 0.7983 - accuracy: 0.6543
Epoch 5/20
440/440 [==============================] - 17s 39ms/step - loss: 0.5735 - accuracy: 0.7811
Epoch 6/20
440/440 [==============================] - 18s 40ms/step - loss: 0.4956 - accuracy: 0.8146
Epoch 7/20
440/440 [==============================] - 18s 40ms/step - loss: 0.4252 - accuracy: 0.8328
Epoch 8/20
440/440 [==============================] - 18s 41ms/step - loss: 0.3397 - accuracy: 0.8714
Epoch 9/20
440/440 [==============================] - 18s 42ms/step - loss: 0.2363 - accuracy: 0.9259
Epoch 10/20
440/440 [==============================] - 18s 42ms/step - loss: 0.140